#### import the things

In [1]:
import pandas as pd
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

idx = pd.IndexSlice

import scipy.stats
import data
raw_data_fol = os.path.join(data.__path__[0], "raw")

In [2]:
from src.data import data_sorting_tools as dst
%load_ext autoreload
%autoreload 2

#### look at tweet data keys

Open one of the jsons to look at key structure and names

In [59]:
query1_file = os.path.join(raw_data_fol, "main_search1_fixed.json")
with open(query1_file) as f:
    query1 = json.load(f)

In [60]:
del query1['query']

In [61]:
query1['0'].keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'extended_entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'matching_rules'])

In [62]:
query1['0']['user'].keys()

dict_keys(['id', 'id_str', 'name', 'screen_name', 'location', 'url', 'description', 'translator_type', 'protected', 'verified', 'followers_count', 'friends_count', 'listed_count', 'favourites_count', 'statuses_count', 'created_at', 'utc_offset', 'time_zone', 'geo_enabled', 'lang', 'contributors_enabled', 'is_translator', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications'])

#### fix truncated tweets

some tweets were truncated due to the way tweepy gathers them. So I had to go through and fix that for all 20000 tweets, and then saved the dictionaries again

In [37]:
q1_fixed = dst.fix_truncated_tweets(query1)

In [63]:
q1_fixed['16'].keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'matching_rules', 'full_text'])

In [64]:
query1['16']['full_text']

'Very useful seminar from @WaverleyCare, who have a base in Glasgow North, this morning on their response to pandemic challenges &amp; supporting people w #HIV &amp; #HepC - seminar running again at lunchtime and later on YouTube, well worth checking out https://t.co/CZkCAfqm5C'

In [47]:
for f in os.listdir(os.path.join(raw_data_fol, "final_main_search")):
    json_file = os.path.join(raw_data_fol, "final_main_search", f)
    with open(json_file, 'r') as j:
        j_dict = json.load(j)
    fixed_dict = dst.fix_truncated_tweets(j_dict)
    
    fname = os.path.join(raw_data_fol, f.replace('.', '_fixed.'))
    with open(fname, 'w') as j:
        json.dump(fixed_dict, j, indent=4)

Rate limit reached. Sleeping for: 735
Rate limit reached. Sleeping for: 751
Rate limit reached. Sleeping for: 745
Rate limit reached. Sleeping for: 749


TweepError: [{'code': 179, 'message': 'Sorry, you are not authorized to see this status.'}]

In [53]:
originals = [f for f in os.listdir(os.path.join(raw_data_fol, "final_main_search"))]

In [50]:
fixed = [f.replace('_fixed', '') for f in os.listdir(raw_data_fol) if "fixed" in f]

In [55]:
(set(originals) - set(fixed))

{'main_search7.json', 'main_search8.json', 'main_search9.json'}

In [56]:
for f in (set(originals) - set(fixed)):
    json_file = os.path.join(raw_data_fol, "final_main_search", f)
    with open(json_file, 'r') as j:
        j_dict = json.load(j)
    fixed_dict = dst.fix_truncated_tweets(j_dict)
    
    fname = os.path.join(raw_data_fol, f.replace('.', '_fixed.'))
    with open(fname, 'w') as j:
        json.dump(fixed_dict, j, indent=4)

In [57]:
fixed = [f.replace('_fixed', '') for f in os.listdir(raw_data_fol) if "fixed" in f]
(set(originals) - set(fixed))

set()

#### decide which keys to put in a dataframe

In [67]:
#this dataframe is just for the text of the tweet, so it's more straightforward
#will keep the same index structure as the numeric df

keepers = [('user','id'), 'text']

df, main_dict = dst.json_to_df(query1_file, keepers, get_media=False, return_dict=True)
df.head()

,user_id,text
1365080049200865280,1135444956145782784,RT @TheKingofReads: “Who gave you HIV?” https:...
1365079654529310721,1042186210943283200,@PozGee04 In the future hiv will just be like ...
1365079624992968708,804135657903824896,@QuasLacrimas @Peter_Nimitz ....of ritual defl...
1365079615115563011,3172127511,A certain boy on Legon campus says he contract...
1365079544022126593,1280964595,A certain boy on Legon campus says he contract...


#### process all the tweet jsons to gather text data

In [68]:
#repeat the process with all the saved json files
keepers = [('user','id'), 'text']

In [69]:
#where all the raw data jsons are kept
os.listdir(raw_data_fol)

['.gitkeep',
 'final_main_search',
 'first_main_search',
 'lifelong_statuses.json',
 'lifelong_user_profile.json',
 'main_search10_fixed.json',
 'main_search11_fixed.json',
 'main_search12_fixed.json',
 'main_search13_fixed.json',
 'main_search14_fixed.json',
 'main_search15_fixed.json',
 'main_search16_fixed.json',
 'main_search17_fixed.json',
 'main_search18_fixed.json',
 'main_search19_fixed.json',
 'main_search1_fixed.json',
 'main_search20_fixed.json',
 'main_search21_fixed.json',
 'main_search22_fixed.json',
 'main_search23_fixed.json',
 'main_search24_fixed.json',
 'main_search25_fixed.json',
 'main_search26_fixed.json',
 'main_search27_fixed.json',
 'main_search28_fixed.json',
 'main_search29_fixed.json',
 'main_search2_fixed.json',
 'main_search30_fixed.json',
 'main_search31_fixed.json',
 'main_search32_fixed.json',
 'main_search33_fixed.json',
 'main_search34_fixed.json',
 'main_search35_fixed.json',
 'main_search36_fixed.json',
 'main_search37_fixed.json',
 'main_search38_f

In [70]:
#go through all the jsons to combine relevant columns of tweets in one dataframe, indexed by tweet ID
all_dfs = []
for f in os.listdir(raw_data_fol):
    if ("main" in f) and (".json" in f):
        df = dst.json_to_df(os.path.join(raw_data_fol, f), keepers, get_media=False, return_dict=False)
        all_dfs.append(df)
        
text_df = pd.concat(all_dfs)

In [135]:
keepers

[('user', 'id'), 'text']

In [139]:
keepers = [('user', 'id'), 'text', 'lang']
#sort through previously collected tweets as well
all_dfs = []
for f in os.listdir(os.path.join(raw_data_fol, 'second_main_search')):
    df = dst.json_to_df(os.path.join(raw_data_fol,'second_main_search', f), keepers, get_media=False, return_dict=False)
    all_dfs.append(df)
        
text_df_2 = pd.concat(all_dfs)

In [140]:
text_df_2.head()

,user_id,text,lang
1363949890598969344,1148573841020428290,大昔の性病とされてきた梅毒に感染する男女が急増中です。HIVやクラミジア、淋病と違い、コンド...,ja
1363949864866971654,1101131404929392641,@isolapine @GrandjeanBenoit a nan mais bien su...,fr
1363949818079375360,1968806107,@N65Michelle @emzanotti @MeghanMcCain I agree-...,en
1363949774060351510,3044766416,RT @clorexiduda: # Esclarecimentos e informaçõ...,pt
1363949759694852107,489662233,RT @DocWoc71: Great 🧵 on #visceralLeishmaniasi...,en


In [141]:
text_df_2 = text_df_2[text_df_2.lang=='en']

In [142]:
text_df_2 = text_df_2.convert_dtypes()

In [143]:
pd.DataFrame(text_df_2.value_counts(subset='text', ascending=False))

,0
text,
RT @jxremivh: To my HIV+ followers: “Your condition is not a confinement” please internalize that phrase and know you’re worthy of love. Yo…,95
"RT @Danez_Smif: Thanks to Alex Hoyt and GQ for the space to talk about HIV, stigma, fuckin, and the first date of that one summer with that…",88
RT @EmekaNwaGuy: PLS RETWEET 🙏\n\nWard divider:40k\nFacial Steamer:22k\nBedside Locker:40k\nOral HIV test kit:3k\nTo order:https://t.co/O0GwsljDW…,77
RT @amandachinda_: How can you be having sex without a condom when you know HIV is real and you are not ready to have a family?,67
RT @Irunnia_: RULE NUMBER 1: \n\nALWAYS KNOW YOUR PARTNER'S HIV STATUS BEFORE ENTERING INTO A RELATIONSHIP!\n\nPLEASE. E GEY WHY!!!\n#keepitsafe…,58
...,...
RT @HIVAIDSupdates: Register now for FREE Work and skills webinars focusing on supporting people living with #HIV who are concerned about #…,1
RT @HIVAIDSupdates: We must treat #HIV stigma like a public health emergency and #UequalsU as an effective response. It's time for everyone…,1
"RT @HIVCenterARC: Join us on Thursday, 3/4, at 9:30 AM EDT for a Grand Rounds presentation by @DrRiveraMindt: ""Health Disparities and Neuro…",1


In [144]:
text_df_2.drop_duplicates('text', keep="first")

,user_id,text,lang
1363949818079375360,1968806107,@N65Michelle @emzanotti @MeghanMcCain I agree-...,en
1363949759694852107,489662233,RT @DocWoc71: Great 🧵 on #visceralLeishmaniasi...,en
1363949727235112961,2151411331,RT @glaad: Join us for the HIV Stigma and Fait...,en
1363949712102076419,17685299,RT @Holbornlolz: Hard Left: We must pursue ZER...,en
1363949692288004103,308788980,So he says what? This is the guy that was in c...,en
...,...,...,...
1363851720321880065,2463473684,RT @TransPHL: The conference covers a wide ran...,en
1363851681021136900,2234067829,Sindh to screen fresh school students for tube...,en
1363851664860598273,16240525,"In 1993, M. Joycelyn Elders became the first B...",en
1363851630718951430,3141853811,"Without addressing mental health, there will b...",en


In [145]:
text_df_2.iloc[0].text

'@N65Michelle @emzanotti @MeghanMcCain I agree-I was teaching HIV/AIDS prevention classes way back when and used Dr… https://t.co/rriyIgjSZ0'

In [ ]:
text_df.drop_duplicates('text', keep="first")

In [71]:
text_df.head()

,user_id,text
1364161520087818243,867033267895226368,Published today: analysis of @Prime_UCL - over...
1364161490668843011,1278251557826502656,There could be 123 000 to 293 000 additional H...
1364161361857679361,1130098020,"RT @dondekojo: Even your HIV status, don't dec..."
1364161322867429376,236806506,"RT @ms_mabila: Teen pregnancy is high, HIV inf..."
1364161232270487553,1331332400836726785,Even though it was a charity. Stevens thought...


In [72]:
text_df.shape

(20000, 2)

In [95]:
#go through all the jsons to combine relevant columns of tweets in one dataframe, indexed by tweet ID
keepers = [('user','id'), 'text']

all_dfs = []
for f in os.listdir(raw_data_fol):
    if ("main" in f) and (".json" in f):
        df = dst.json_to_df(os.path.join(raw_data_fol, f), keepers, get_media=False, return_dict=False)
        all_dfs.append(df)
        
alt_text_df = pd.concat(all_dfs)

In [97]:
alt_text_df.head()

,user_id,"(user, id)",text
1364161520087818243,867033267895226368,Published today: analysis of @Prime_UCL - over...,Published today: analysis of @Prime_UCL - over...
1364161490668843011,1278251557826502656,There could be 123 000 to 293 000 additional H...,There could be 123 000 to 293 000 additional H...
1364161361857679361,1130098020,NaN,"RT @dondekojo: Even your HIV status, don't dec..."
1364161322867429376,236806506,NaN,"RT @ms_mabila: Teen pregnancy is high, HIV inf..."
1364161232270487553,1331332400836726785,Even though it was a charity. Stevens thought...,Even though it was a charity. Stevens thought...


In [99]:
alt_text_df.columns

Index(['user_id', ('user', 'id'), 'text'], dtype='object')

In [117]:
alt_text_df.rename(columns={('user', 'id'):'trunc_text'}, inplace=True)

In [118]:
alt_text_df.columns

Index(['user_id', 'trunc_text', 'text'], dtype='object')

In [106]:
alt_text_df.head()

,user_id,trunc_text,text
1364161520087818243,867033267895226368,Published today: analysis of @Prime_UCL - over...,Published today: analysis of @Prime_UCL - over...
1364161490668843011,1278251557826502656,There could be 123 000 to 293 000 additional H...,There could be 123 000 to 293 000 additional H...
1364161361857679361,1130098020,NaN,"RT @dondekojo: Even your HIV status, don't dec..."
1364161322867429376,236806506,NaN,"RT @ms_mabila: Teen pregnancy is high, HIV inf..."
1364161232270487553,1331332400836726785,Even though it was a charity. Stevens thought...,Even though it was a charity. Stevens thought...


In [115]:
alt_text_df = alt_text_df.convert_dtypes()
alt_text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1364161520087818243 to 1364931661583048707
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     20000 non-null  Int64 
 1   (user, id)  15704 non-null  string
 2   text        20000 non-null  string
dtypes: Int64(1), string(2)
memory usage: 644.5 KB


In [119]:
un_truncs = alt_text_df[pd.isnull(alt_text_df.trunc_text)==False]
un_truncs.head()

,user_id,trunc_text,text
1364161520087818243,867033267895226368,Published today: analysis of @Prime_UCL - over...,Published today: analysis of @Prime_UCL - over...
1364161490668843011,1278251557826502656,There could be 123 000 to 293 000 additional H...,There could be 123 000 to 293 000 additional H...
1364161232270487553,1331332400836726785,Even though it was a charity. Stevens thought...,Even though it was a charity. Stevens thought...
1364161201291153414,150301804,SCORA Standing Committee on Sexual &amp; Repro...,SCORA Standing Committee on Sexual &amp; Repro...
1364161184505737217,1317091162159747075,"many females are HIV+, we wish you well, bless...","many females are HIV+, we wish you well, bless..."


In [120]:
un_truncs.iloc[0].trunc_text

'Published today: analysis of @Prime_UCL - over 50% of women with HIV aged 45-60 reported moderate/extreme pain. Increased prevalence in perimenopause. Pain assoc with insomnia + depression highlighting importance of eliciting pain history. Read more: https://t.co/woqgA0MGku https://t.co/vPNi0ORxse'

In [124]:
un_truncs.iloc[0].text[:10]

'Published '

In [126]:
un_truncs.iloc[0].trunc_text == un_truncs.iloc[0].text

True

In [129]:
un_truncs = un_truncs.assign(Same = lambda x: x['trunc_text'] == x['text'])

In [131]:
un_truncs.head()

,user_id,trunc_text,text,Same
1364161520087818243,867033267895226368,Published today: analysis of @Prime_UCL - over...,Published today: analysis of @Prime_UCL - over...,True
1364161490668843011,1278251557826502656,There could be 123 000 to 293 000 additional H...,There could be 123 000 to 293 000 additional H...,True
1364161232270487553,1331332400836726785,Even though it was a charity. Stevens thought...,Even though it was a charity. Stevens thought...,True
1364161201291153414,150301804,SCORA Standing Committee on Sexual &amp; Repro...,SCORA Standing Committee on Sexual &amp; Repro...,True
1364161184505737217,1317091162159747075,"many females are HIV+, we wish you well, bless...","many females are HIV+, we wish you well, bless...",True


In [130]:
un_truncs[un_truncs.Same==False]

,user_id,trunc_text,text,Same


In [133]:
pd.DataFrame(un_truncs.value_counts(subset='trunc_text', ascending=False))

,0
trunc_text,
Giving the girl HIV because her man clearly wasn’t paying enough attention to her then MAKING the girl pick up her hiv medication from his family store! SICK! https://t.co/BB4yKczIjW,412
"No pride for some of us without liberation for all of us.\n\nMarsha P. Johnson spent her life advocating for LGBTQ people of color, LGBTQ youth experiencing homelessness, and people living with HIV and AIDS.\n\nThank you, Marsha. #BlackHistoryMonth https://t.co/S9MDdKnD35",391
I hope someone is showing Baba this nation appreciation. History has no blank pages. I served on the board of SANAC the highest advisory body on HIV under Nxamalala. Changed lives &amp; crushed stigma. He received criticism for appointing me and was not fazed 🙏🏾 #IBenefitedUnderZuma,261
Ritchie and Ash lying in a hospital bed together is a beautiful moment of love.\n\nIt’s reminiscent of Gideon Mendel’s photo series where he visited an AIDS ward to show the loving relationships between the men who were dying and their partners.\n\nTo humanise HIV.\n\n#ItsASin https://t.co/MLv3sGV8ik,242
A certain boy on Legon campus says he contracted HIV from a one night stand now he’s going around spreading the virus. This is not funny and should be taken serious. Please try and protect yourself. I know most of you don’t like to get tested so try and stay with one partner.,239
...,...
"For queer folks, the HIV/AIDS epidemic was deeply traumatizing. Russell Davies' new series ""It's a Sin,"" a show depicting, and often celebrating, the folks who lived and died through the 80s, feels like it's right on time.\n\n#CultureIsPower\n#HIVepidemic\nhttps://t.co/1cZoYbS8Kh",1
"Twitter,facebook,linkedin,flickr,youtube,weibo,instangram,wechat,tiktok,zalo:7 billions people:go to:https://t.co/73wWYwdVsP \nand all the world test HIV now before infected HIV your wife or your husbands,all in my many USB at every where. continue,,,, https://t.co/T67tiEEglN",1
"For our second interview of the season, we had the privilege of interviewing Dr. Peter Rebeiro! We discuss the importance of infectious disease epidemiology, the HIV Care Continuum, health disparities of HIV, and mental health of HIV. #epitwitter #MedTwitter https://t.co/MqvKelIZIk",1


#### clean up the dataframe & save it as a pickle file

In [73]:
#multi index the df on tweet id and user id
text_df.set_index('user_id', drop=True, append=True, inplace=True)
text_df.index.rename(['tid', 'uid'], inplace=True)

#show any true duplicates, based off of unique tweet id (tid)
text_df[text_df.index.get_level_values('tid').duplicated(keep=False)]

,,text
tid,uid,


In [74]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 20000 entries, (1364161520087818243, 867033267895226368) to (1364931661583048707, 951148804564307969)
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20000 non-null  object
dtypes: object(1)
memory usage: 1.8+ MB


In [75]:
#convert dtypes to most obvious type
text_df = text_df.convert_dtypes()
text_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 20000 entries, (1364161520087818243, 867033267895226368) to (1364931661583048707, 951148804564307969)
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20000 non-null  string
dtypes: string(1)
memory usage: 1.8 MB


remove duplicates due to retweets

In [83]:
text_df.shape

(20000, 1)

In [84]:
text_df.drop_duplicates('text', keep=False).sort_values('text')

,,text
tid,uid,
1363927154572939266,1257853784945946624,"""#Adherence is the single biggest factor in wh..."
1364976477670932484,38105260,"""#Optimism 😊remains important. Being diagnosed..."
1364963991550898194,165339820,"""'The concept of the campaign is to show that ..."
1364937012286021632,405170497,"""... they say that SARS-CoV-2 is not a retrovi..."
1365043923392081935,2903979971,"""Adult authors and publishers seem more intere..."
...,...,...
1364967393575993356,747855054854561792,🤑 Updated: Funding List🤑 We've got a residenc...
1364977219332956165,256455823,🤔 How much do you know about HIV? 🤔 Take 2 min...
1363060840992227329,256455823,🤔 How much do you know about HIV? 🤔 Take 2 min...


In [80]:
dupes = text_df.drop_duplicates('text', keep=False).sort_values('text')
dupes

,,text
tid,uid,
1363927154572939266,1257853784945946624,"""#Adherence is the single biggest factor in wh..."
1364976477670932484,38105260,"""#Optimism 😊remains important. Being diagnosed..."
1364963991550898194,165339820,"""'The concept of the campaign is to show that ..."
1364937012286021632,405170497,"""... they say that SARS-CoV-2 is not a retrovi..."
1365043923392081935,2903979971,"""Adult authors and publishers seem more intere..."
...,...,...
1364967393575993356,747855054854561792,🤑 Updated: Funding List🤑 We've got a residenc...
1364977219332956165,256455823,🤔 How much do you know about HIV? 🤔 Take 2 min...
1363060840992227329,256455823,🤔 How much do you know about HIV? 🤔 Take 2 min...


In [134]:
text_df.drop_duplicates('text', keep="first")

,,text
tid,uid,
1364161520087818243,867033267895226368,Published today: analysis of @Prime_UCL - over...
1364161490668843011,1278251557826502656,There could be 123 000 to 293 000 additional H...
1364161361857679361,1130098020,"RT @dondekojo: Even your HIV status, don't dec..."
1364161322867429376,236806506,"RT @ms_mabila: Teen pregnancy is high, HIV inf..."
1364161232270487553,1331332400836726785,Even though it was a charity. Stevens thought...
...,...,...
1364932183023095815,3267499909,@eliowa @AlexDoser1 I thought the efficacy in ...
1364932040156659715,4030326073,In the 55th plenary meeting of the 75th #UNGA ...
1364931957977735172,1327999757634199552,with the LGBT poll thing yesterday I didn't se...


In [91]:
RT_counts = pd.DataFrame(text_df.value_counts(subset='text', ascending=False))

In [92]:
RT_counts[RT_counts[0] > 1]

,0
text,
Giving the girl HIV because her man clearly wasn’t paying enough attention to her then MAKING the girl pick up her hiv medication from his family store! SICK! https://t.co/BB4yKczIjW,412
"No pride for some of us without liberation for all of us.\n\nMarsha P. Johnson spent her life advocating for LGBTQ people of color, LGBTQ youth experiencing homelessness, and people living with HIV and AIDS.\n\nThank you, Marsha. #BlackHistoryMonth https://t.co/S9MDdKnD35",391
I hope someone is showing Baba this nation appreciation. History has no blank pages. I served on the board of SANAC the highest advisory body on HIV under Nxamalala. Changed lives &amp; crushed stigma. He received criticism for appointing me and was not fazed 🙏🏾 #IBenefitedUnderZuma,261
Ritchie and Ash lying in a hospital bed together is a beautiful moment of love.\n\nIt’s reminiscent of Gideon Mendel’s photo series where he visited an AIDS ward to show the loving relationships between the men who were dying and their partners.\n\nTo humanise HIV.\n\n#ItsASin https://t.co/MLv3sGV8ik,242
A certain boy on Legon campus says he contracted HIV from a one night stand now he’s going around spreading the virus. This is not funny and should be taken serious. Please try and protect yourself. I know most of you don’t like to get tested so try and stay with one partner.,239
...,...
The fight against HIV has always been a fight. Lives were saved by those who wouldn’t be silenced or ignored by those unaffected.\n\nToday the fight is different. But — while we’ve come a long way in the fight against HIV — it’s not over. In the UK or globally. \n\n#ItsASin https://t.co/ZY3JKn7igM,2
"Within 3 years of #opencontracting reforms the number of people receiving treatment for conditions like HIV grew from 50,000 to 113,000, without any increase in budget. Read how Ukraine reformed medical procurement: https://t.co/XqB2HShkgN https://t.co/fgyr2u9iHS",2
Through the success of @russelldavies63 'It’s A Sin' we’ve seen a remarkable swell in discussions about HIV and representations of HIV across various medias. So here's our Staff Top 12 for Song that also spotlight HIV as a central narrative. What else should be on the list? https://t.co/ByzldGXGtL,2


In [94]:
np.sum(RT_counts[RT_counts[0] > 1])

0    13153
dtype: int64

In [33]:
#final structure
text_df.head()

,,text
tid,uid,
1363949818079375360,1968806107,@N65Michelle @emzanotti @MeghanMcCain I agree-...
1363949759694852107,489662233,RT @DocWoc71: Great 🧵 on #visceralLeishmaniasi...
1363949727235112961,2151411331,RT @glaad: Join us for the HIV Stigma and Fait...
1363949712102076419,17685299,RT @Holbornlolz: Hard Left: We must pursue ZER...
1363949692288004103,308788980,So he says what? This is the guy that was in c...


In [39]:
#save the dataframe as a pickle file to be loaded into other notebooks
pd.to_pickle(text_df, os.path.join(data.__path__[0], "compiled", "first_6000_text.pkl"))

In [37]:
#save the tweet IDs of the non-english tweets to remove them from the numeric dataframe as well
np.save(os.path.join(data.__path__[0], "compiled", "first_6000_non_eng_ind.npy"), drop_tweets_id)

In [40]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4567 entries, (1363949818079375360, 1968806107) to (1363851623584444420, 35814608)
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4567 non-null   string
dtypes: string(1)
memory usage: 438.1 KB
